Load local model

In [2]:
# Local
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from libs.core.tests.unit_tests.output_parsers.test_openai_tools import message

ds_chat = ChatOllama(model="deepseek-r1:7b")

C:\Users\guosh\AppData\Local\Temp\ipykernel_38700\3086055942.py:8: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  ds_chat = ChatOllama(model="deepseek-r1:7b")


Maintain message list

In [3]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful assistant. Answer all questions to the best of your ability. Answer in English.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | ds_chat
mes_list=    {
        "messages": [
            HumanMessage(
                content="Translate from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
ai_msg = chain.invoke(mes_list)
print(ai_msg.content)

<think>
Okay, so the user asked me "I love programming." and I translated it into French as "J'adore la programmation." Now they're following up with "What did you just say?" Hmm, maybe they want a clearer or more natural translation.

Let me think about why they might be asking this. They could be testing my language skills or perhaps interested in how to express similar sentences differently. Maybe they found the first response too literal and are looking for something smoother.

I should acknowledge that I translated "love" as "adore," which is common, but sometimes people use "apprends" for learning or "passionne" for passion. Wait, no, "passione" is more like passion in French. Maybe "J'apprends la programmation." sounds a bit less formal and more conversational.

So I'll explain that both translations are correct—using "adore" is standard, but offering an alternative might make it sound friendlier or more natural to them. That way, they can choose the one that fits better in thei